<a href="https://colab.research.google.com/github/YurleyG/Analitica-de-negocios/blob/main/Reto_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Descripción de las variables**
1. College: indica si la persona posee un titulo universitario (1) o no(0). *Es la variable a determinar*
2. Business: representa el sector del negocio siendo Hardware: 0 y Software: 1

Y las demás variables socio-económicas a analizar tales como edad, sexo, años, certificados, feedback, salario y NPS

0. Se procede con la carga de las librerias de trabajo

In [ ]:
import numpy as np
import pandas as pd

#Para implementación y evaluación del modelo
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix

#Para la carga automática de los archivos desde el drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


1. Se procede con la carga de la base de datos

In [ ]:
nxl="/content/drive/MyDrive/Colab Notebooks/2. TechSalesReport_Int_L.xlsx"
XDB=pd.read_excel(nxl,sheet_name=0)
XDB.head(20) #Solo se muestran 20 datos
#Seleccionamos las variables
XD=XDB[["Business","Age","Female","Years","Certificates","Feedback","Salary","NPS"]]
yd=XDB[["College"]]


2. Se procede con la implementación del modelo

In [ ]:
# Hardware 0, Software 1
XD['Business'] = XD['Business'].map({'Hardware': 0, 'Software': 1})

mar=DecisionTreeClassifier(criterion='gini', max_depth=4)
mar.fit(XD,yd)

#Se procede con el despliegue del arbol
from sklearn.tree import export_graphviz #Para exportar
from pydotplus import graph_from_dot_data #Crear el gráfico

vs=["Business","Age","Female","Years","Certificates","Feedback","Salary","NPS"]

dot_data=export_graphviz(mar,feature_names=vs)
graph=graph_from_dot_data(dot_data)
graph.write_png('ArbolPreApr.png')

<ipython-input-18-36746554fb65>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  XD['Business'] = XD['Business'].map({'Hardware': 0, 'Software': 1})


True

4. Número de personas inscritas y las no inscritas

In [ ]:
# Contar el número de personas inscritas y no inscritas
inscritos = XDB['College'].value_counts()[1]
no_inscritos = XDB['College'].value_counts()[0]

# Imprimir los resultados
print(f"Número de personas inscritas: {inscritos}")
print(f"Número de personas no inscritas: {no_inscritos}")

Número de personas inscritas: 4470
Número de personas no inscritas: 17520


<ipython-input-14-d533849e36f7>:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  inscritos = XDB['College'].value_counts()[1]
<ipython-input-14-d533849e36f7>:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  no_inscritos = XDB['College'].value_counts()[0]


5. Desempeño del modelo

In [ ]:
ydp=mar.predict(XD)
cm=confusion_matrix(yd,ydp)
print(cm)

VN=cm[0,0]; FP=cm[0,1]; FN=cm[0,1]; VP=cm[1,1]

Exactitud=(VP+VN)/(VP+VN+FP+FN) #Funcionamiento general del modelo
print("La exactitud es:\=n",Exactitud)
Sensibilidad=VP/(VP+FN) #Cómo se comportó el modelo con los aprobados
print('la Sensibilidad es:\=n',Sensibilidad)
Especificidad=VN/(VN+FP) #Cómo se comportó el modelo con los negados
print('la Especificidad es:\=n',Especificidad)
Precisión=VP/(VP+FP) #Cuál es el acierto clasificando los positivos
print('la Precisión es:\=n',Precisión)
TasaError=(FP+FN)/(VP+VN+FP+FN) #Equivocación general
print('la TasaError es:\=n',TasaError)
PredNeg=VN+FP #Cómo funciona el modelo negando creditos
print('la PredNeg es:\=n',PredNeg)

[[  114  4356]
 [   69 17451]]
La exactitud es:\=n 0.6684553031167941
la Sensibilidad es:\=n 0.8002476269087908
la Especificidad es:\=n 0.025503355704697986
la Precisión es:\=n 0.8002476269087908
la TasaError es:\=n 0.33154469688320587
la PredNeg es:\=n 4470


5. Creamos el informe para los que están inscritos

In [ ]:
# Realizar la predicción utilizando el modelo entrenado (mar)
ydp = mar.predict(XD)

# Agregar las predicciones como una nueva columna al DataFrame XDB2
XDB['College_Predicción'] = ydp

# Mostrar las predicciones junto con otros datos relevantes
print(XDB[['College', 'College_Predicción']])

#Crear el informe con el College
dfp=pd.DataFrame(np.column_stack((XD,ydp)))
dfp.columns=["Business","Age","Female","Years","Certificates","Feedback","Salary","NPS","College"]
dfp.head(10)
dfp.to_excel("InformeCollege.xlsx")

      College College_Predicción
0         Yes                Yes
1         Yes                Yes
2         Yes                Yes
3         Yes                Yes
4         Yes                Yes
...       ...                ...
21985     Yes                Yes
21986     Yes                Yes
21987     Yes                Yes
21988     Yes                Yes
21989     Yes                Yes

[21990 rows x 2 columns]


**Análisis de resultados**

Para la implementación del modelo de Arbol de decisión, se tomaron un total de 9 variables, 8 de entrada entre las que se destacan:Business,Age,Female,Years,Certificates,Feedback,Salary y NPS, y una variable de salida que permite evaluar si la persona analizada posee un titulo universitario. A partir de la creación del arbol de decisión se pudo evidenciar que la variable menos relevante fue el salario y que no se presentan nodos puros sin embargo hay ciertos con indices muy bajos como 0.075 y 0.138. Para la construcción del modelo se tuvieron un total de 21990 personas divididos en 17520 con titulo y 4470 que no. Por medio de la ejecución del modelo se tuvo una exactitud cercana al 70%, lo que indica la eficacia del modelo para pronosticar. Cabe recalcar, que el coefciente de sensibilidad estuvo en 80% lo que reafirma el buen funcionamiento del modelo.